## First approach for meassurement

For doing some EDA I will use a simple CNN with two convultion blocks and a flatten dense layer with a softmax activation as ouput

In [1]:
import cv2
import gc
import os
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import keras.backend as K
import tensorflow as tf

from sklearn.datasets import load_files
from glob import glob
from PIL import Image

from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.applications import InceptionV3, VGG16
from keras.regularizers import l1_l2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

%matplotlib inline
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/gpu:0']

As shown on the description we have 30 unique labels under the Animal column. There a chance of applying the keras common structure for DataAugmentation.

For now I´m going to jump directly into the mood.

In [ ]:
# Variables
TRAIN_PATH = 'dataset/train/'
TEST_PATH = 'dataset/test1/'
EPOCHS = 2000
IMG_PATH =  TRAIN_PATH+str(train.Image_id[0])
IMG_WIDTH = 224
IMG_HEIGHT = 224
CHANNELS = 3
INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, CHANNELS)
NB_CLASSES = 30
REG = 1e-3

In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

        
def read_img(img_path, img_size=(IMG_WIDTH, IMG_HEIGHT)):
    """
    Transforms an image from the given path to an array of the size given.
    
    The ouput will be the image with a shape of (img_width, img_height, 3).
    Its assumed that images are RGB.
    """
    img = cv2.imread(img_path)
    img = cv2.resize(img, img_size)
    return img        

def get_labels(df):
    label_cols = list(set(train.columns) - set(['Image_name']))
    label_cols.sort()
    return df.iloc[0][2:].index[train.iloc[0][2:] == 1]

def load_pictures(directory, dataset):
    label_enconder = LabelEncoder()
    dataset.Animal = label_enconder.fit_transform(dataset.Animal)
    
    y = []
    X = []
    for sample in log_progress(dataset.values, every=10):
        X.append(read_img(directory + sample[0]))
        y.append(sample[1]) 
    return X, y

def load_test_pictures(directory, dataset):
    X = []
    for sample in log_progress(dataset.values, every=10):
        X.append(read_img(directory + sample[0]))
    return X

def create_datagen():
    return ImageDataGenerator(
        rescale = 1./255,
        horizontal_flip = False,
        vertical_flip = True,
        shear_range = 5.,
        fill_mode = "nearest",
        zoom_range = 0.2,
        width_shift_range = 0.1,
        height_shift_range=0.2,
        rotation_range=20)

def create_val_datagen():
    return ImageDataGenerator(
        rescale = 1./255,
        horizontal_flip = False,
        vertical_flip = True,
        fill_mode = "nearest",
        width_shift_range = 0.1,
        height_shift_range=0.2)

def normailize_data(data):
    return np.array(data).astype('float32') / 255.

In [ ]:
class CustomCNN:
    @staticmethod
    def build(input_shape = (IMG_WIDTH, IMG_WIDTH, CHANNELS), classes=NB_CLASSES):                             
        model = Sequential()
        
        model.add(BatchNormalization(input_shape=input_shape))
        model.add(Conv2D(32, (3, 3), padding='same', activation='relu' ))
        model.add(Conv2D(32, (3, 3),activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))
        
        model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))
        
        
        model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        
        model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(256, (3, 3), activation='relu'))
        model.add(Conv2D(256, (3, 3), activation='relu'))        
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))        
        
        model.add(Flatten())
        model.add(BatchNormalization())
        model.add(Dense(1024, activation='relu'))
        model.add(Dense(1024, activation='relu'))
        model.add(Dense(classes, activation='softmax'))
        
        return model

class LeNet:
    @staticmethod
    def build(input_shape, classes):
        model = Sequential()
        
        model.add(Conv2D(20, kernel_size=5, padding='same', activation='relu', input_shape=input_shape))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        model.add(Dropout(0.5))
        
        model.add(BatchNormalization())
        model.add(Conv2D(50, kernel_size=5, padding='same',activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))        
        model.add(Dropout(0.5))
        
        model.add(Flatten())
        model.add(Dense(500, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(classes, activation='softmax'))
        
        return model
    
class InceptionV3Ext:
    @staticmethod
    def build(input_shape, classes):
        v3_model = InceptionV3(weights = "imagenet", include_top=False, input_shape = INPUT_SHAPE)
        for layer in v3_model.layers: layer.trainable = False
            
            
        #Adding custom Layers 
        x = v3_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(1024, activation='relu')(x)

        predictions = Dense(classes, activation="softmax")(x)

        # creating the final model 
        return Model(input = v3_model.input, output = predictions)

class VGG16Ext:
    @staticmethod
    def build(input_shape, classes):
        model = VGG16(weights = "imagenet", include_top=False, input_shape = INPUT_SHAPE)
        for layer in model.layers: layer.trainable = False
            
            
        #Adding custom Layers 
        model_ext = model.output
        model_ext = Dense(1024, activation='relu')(model_ext)
        model_ext = Dense(1024, activation='relu')(model_ext)
        
        preductions = Dense(classes, activation="softmax")(model_ext)

        # creating the final model 
        return Model(inputs = model.input, outputs = predictionsictions)    

class BasicCNN:
    @staticmethod
    def build(input_shape = (IMG_WIDTH, IMG_WIDTH, CHANNELS), classes=NB_CLASSES):
        model = Sequential()
        model.add(Dense(512, activation='relu', input_shape=input_shape))
        model.add(Dense(512, activation='relu'))
        model.add(Dense(classes, activation='softmax'))        

In [ ]:
gc.collect()

X, y = load_pictures(TRAIN_PATH, train)

In [ ]:
plt.imshow(X[0], interpolation='nearest')
plt.show()

### Genererating de final training dataset

In [ ]:
X = normailize_data(X)

Xtrain, Xvalid, ytrain, yvalid = train_test_split(X, y, test_size=0.25)

ytrain = to_categorical(ytrain, NB_CLASSES)
yvalid = to_categorical(yvalid, NB_CLASSES)

print('Xtrain shape: ', Xtrain.shape)
print('Xvalid shape: ', Xvalid.shape)
print('ytrain shape: ', ytrain.shape)
print('yvalid shape: ', yvalid.shape)

In [ ]:
plt.imshow(Xtrain[7], interpolation='nearest')
plt.title(ytrain[7])
plt.show()

print(type(ytrain[0][0]))

In [ ]:
plt.imshow(Xvalid[7], interpolation='nearest')
plt.title(yvalid[7])
plt.show()

## Custom net... or something yacnn bla bla

3 simple block of

Input (128, 128, 3)
Conv2D 32 3,3
Conv2D 32 3,3
Pooling
Dropout

Conv2D 64 3,3
Conv2D 64 3,3
Pooling
Dropout

Conv2D 128 3,3
Conv2D 128 3,3
Pooling
Dropout

Flatten
Dense 512
Dropout
Dense - classes

In [ ]:
gc.collect()

#model = CustomCNN.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
#model = LeNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
model = InceptionV3Ext.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
#model = VGG16Ext.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
model.summary()

In [ ]:
sgd_optimizer = SGD(lr=0.00001, decay=1e-3, momentum=0.9, nesterov=False)
adam_optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [ ]:
with tf.device('/gpu:0'):    
    model.compile(optimizer=adam_optimizer, 
                  loss='categorical_crossentropy', 
                  metrics=['acc'])
    
    early_stops = EarlyStopping(patience=20, monitor='val_acc', verbose=1)
    checkpointer = ModelCheckpoint(filepath='weights.best.eda.hdf5', verbose=1, save_best_only=True)
    
    callbacks = [checkpointer, early_stops]
    
    h = model.fit(Xtrain, ytrain, 
                  validation_data=(Xvalid, yvalid), 
                  epochs=EPOCHS, 
                  batch_size=10, 
                  callbacks=callbacks, 
                  verbose=1)

In [ ]:
# list all data in history
print(h.history.keys())
# summarize history for accuracy
plt.plot(h.history['acc'])
plt.plot(h.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

model.save_weights("final_ouput.hdf5")

In [ ]:
test.head()

In [ ]:
labels_col = train.Animal.unique()

In [ ]:
test_img = load_test_pictures(TEST_PATH, test)

X_test = normailize_data(test_img)     

del test_img
gc.collect()


model.load_weights('final_ouput.hdf5')
pred_test = model.predict(X_test,verbose=1)

In [ ]:
pred_train = model.predict(X, verbose=1)

In [ ]:
pd.options.display.float_format = '{:,.8f}'.format

subm = pd.DataFrame()
subm['image_id'] = test.Image_id
label_df = pd.DataFrame(data=pred_test, columns=labels_col)
subm = pd.concat([subm, label_df], axis=1)
subm = subm.set_index('image_id')
subm = subm.reindex(sorted(subm.columns), axis=1)
subm.to_csv('submit.csv', index=True)
subm.head(5)

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
print('Log_loss: {}'.format(log_loss(pred_test, y_predict)))